In [1]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('by_t1.csv')
df

,Time id,Material,Quantity,Total_Value,Material_Id
0,1,C24-100504,3.0,38991.70,1311
1,1,C24-700110,28.0,60159.68,1421
2,1,C29-700105,2.0,28757.51,1610
3,1,C37-500419,4.0,51712.67,1755
4,1,C87-200404,3.0,146492.80,4756
...,...,...,...,...,...
382,38,S06-010053,9.0,73800.00,6464
383,38,S10-020027,116.0,75397.39,7263
384,38,S23-170301,111.0,120978.90,30765
385,38,S23-240014,150.0,82446.74,8710


In [3]:
import pandas as pd

# Assuming your DataFrame is named df

# Filter the DataFrame for Time ids 37 and 38
filtered_df = df[df['Time id'].isin([37, 38])]

# Display the resulting DataFrame
print(filtered_df)

     Time id      Material  Quantity  Total_Value  Material_Id
366       37    C04-400301       6.0     46513.98        32814
367       37    C24-100504       3.0     40315.89         1311
368       37    C24-700110      20.0     45523.20         1421
369       37    C29-500264       1.0     11034.13        12481
370       37    C29-700105       2.0     28880.00         1610
371       37  CB-LS01-0700     105.0     67095.00        25056
372       37    S02-060500       7.0    200787.32         6345
373       37    S06-010053       5.0     41000.00         6464
374       37    S10-020027     105.0     68248.65         7263
375       37    S23-170301     107.0    116619.30        30765
376       37    S23-240014     124.0     68189.19         8710
377       38    C04-400301       6.0     47070.00        32814
378       38    C24-700110      24.0     54627.84         1421
379       38    C29-500264       8.0     88273.04        12481
380       38    C29-700105       1.0     14440.00      

In [4]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 1311
Training SimpleRNN model for Material ID: 1311


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
MAE: 0.11217408875624339, MAPE: inf, R-squared: 0.8303691227470049
Training GRU model for Material ID: 1311


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
MAE: 0.1389851172765096, MAPE: inf, R-squared: 0.7450461159692834
Training LSTM model for Material ID: 1311


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 919ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
MAE: 0.14056103229522704, MAPE: inf, R-squared: 0.7375365350641317
Training LSTM model for Material ID: 32814


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
MAE: 0.1831642806529999, MAPE: inf, R-squared: 0.5571546955298454
   Time id      Material  Quantity  Total_Value Material_Id      Model  \
0       37    C24-100504       3.0     40315.89        1311  SimpleRNN   
1       37    C24-100504       3.0     40315.89        1311        GRU   
2       37    C24-100504       3.0     40315.89        1311       LSTM   
3       37    C24-700110      24.0     54627.84        1421  SimpleRNN   
4       37    C24-700110      24.0     54627.84        1421        GRU   
5       37    C24-700110      24.0     54627.84        1421       LSTM   
6       37    C29-700105       1.0     14440.00        1610  SimpleRNN   
7       37    C29-700105       1.0     14440.00        1610        GRU   
8       37    C29-700105       1.0     14440.00        1610       LSTM   
9       37    C37-500419       1.0     15447.40        1755  SimpleRNN   
10      37    C37-500419       1.0     154

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\4108763989.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [5]:
forecast_df.to_csv('by1.csv', index=False)


In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 36
forecast_time_id = 37

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 1311
Evaluating SimpleRNN model for Material ID: 1311


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
MAPE: 17.515603701273598
Evaluating GRU model for Material ID: 1311


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
MAPE: 2.41244633992513
Evaluating LSTM model for Material ID: 1311


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
MAPE: 71.10001246134439
Evaluating LSTM model for Material ID: 32814


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
MAPE: 61.22659047444662
   Material_Id      Model        MAPE
0         1311  SimpleRNN   17.515604
1         1311        GRU    2.412446
2         1311       LSTM    6.550455
3         1421  SimpleRNN  175.582066
4         1421        GRU  155.283451
5         1421       LSTM  146.044865
6         1610  SimpleRNN    1.988286
7         1610        GRU   24.535763
8         1610       LSTM   43.234468
9         6345  SimpleRNN   14.305871
10        6345        GRU   28.668785
11        6345       LSTM   34.271376
12        6464  SimpleRNN  133.031559
13        6464        GRU   99.882011
14        6464       LSTM   92.830505
15        7263  SimpleRNN    2.899257
16        7263        GRU    1.894589
17        7263       LSTM    2.002505
18        8710  SimpleRNN   19.402842
19        8710        GRU   22.172011
20        8710       LSTM   22.104786
21       12481  SimpleRNN  316.774464
22       12481        GRU  335.591841
23       12481       LSTM

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\472838995.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [7]:
forecast_df.to_csv('by2.csv', index=False)


In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    return mae, mape, r2

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Time id', 'Material', 'Quantity', 'Total_Value', 'Material_Id', 'Model', 'Forecasted', 'MAE', 'MAPE', 'R2'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]


    # Skip evaluation if there is no training data
    if train_data.empty:
        print(f"No training data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Training {model_type} model for Material ID: {material_id}")
        
        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)
        
        # Forecast for the next time id (Time id 37)
        forecasted_values = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecasted_values = scaler.inverse_transform(forecasted_values.reshape(-1, 1)).flatten()
        forecast = forecasted_values[0]

        # Evaluate the model for the current Material Id
        mae, mape, r2 = evaluate_model(model, X_train, y_train)
        print(f"MAE: {mae}, MAPE: {mape}, R-squared: {r2}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Time id': forecast_time_id,
            'Material': material_data['Material'].iloc[-1],
            'Quantity': material_data['Quantity'].iloc[-1],
            'Total_Value': material_data['Total_Value'].iloc[-1],
            'Material_Id': material_id,
            'Model': model_type,
            'Forecasted': forecast,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2
        }, ignore_index=True)

# Display the forecast DataFrame
print(forecast_df)


Evaluating models for Material ID: 1311
Training SimpleRNN model for Material ID: 1311


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
MAE: 0.07602484753498664, MAPE: inf, R-squared: 0.9186951409109081
Training GRU model for Material ID: 1311


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
MAE: 0.1465919017791748, MAPE: inf, R-squared: 0.7052053162897576
Training LSTM model for Material ID: 1311


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
MAE: 0.05326055648264946, MAPE: inf, R-squared: 0.9576412538331998
Training LSTM model for Material ID: 32814


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
MAE: 0.15393843008326247, MAPE: inf, R-squared: 0.6515822805078904
   Time id      Material  Quantity  Total_Value Material_Id      Model  \
0       38    C24-100504       3.0     40315.89        1311  SimpleRNN   
1       38    C24-100504       3.0     40315.89        1311        GRU   
2       38    C24-100504       3.0     40315.89        1311       LSTM   
3       38    C24-700110      24.0     54627.84        1421  SimpleRNN   
4       38    C24-700110      24.0     54627.84        1421        GRU   
5       38    C24-700110      24.0     54627.84        1421       LSTM   
6       38    C29-700105       1.0     14440.00        1610  SimpleRNN   
7       38    C29-700105       1.0     14440.00        1610        GRU   
8       38    C29-700105       1.0     14440.00        1610       LSTM   
9       38    C37-500419       1.0     15447.40        1755  SimpleRNN   
10      38    C37-500419       1.0     15

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:12: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\3232560010.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [9]:
forecast_df.to_csv('by3.csv', index=False)


In [10]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense
import numpy as np
import pandas as pd

# Define the evaluate_model function with MAPE
def evaluate_model(y_pred, y_actual):
    mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
    return mape

# Define train and test periods
train_start_id = 25
train_end_id = 37
forecast_time_id = 38

# Create an empty DataFrame to store forecasted values
forecast_df = pd.DataFrame(columns=['Material_Id', 'Model', 'MAPE'])

# Define a function to build and train RNN model
def build_and_train_model(X_train, y_train, model_type):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material id
for material_id in df['Material_Id'].unique():
    print(f"Evaluating models for Material ID: {material_id}")
    
    # Filter data for the current material id
    material_data = df[df['Material_Id'] == material_id]
    
    # Filter training data
    train_data = material_data[(material_data['Time id'] >= train_start_id) & (material_data['Time id'] <= train_end_id)]
    test_data = material_data[material_data['Time id'] == forecast_time_id]

    # Skip evaluation if there is no training or test data
    if train_data.empty or test_data.empty:
        print(f"No training or test data available for Material ID: {material_id}. Skipping evaluation.")
        continue

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Evaluate SimpleRNN, GRU, and LSTM models
    for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
        print(f"Evaluating {model_type} model for Material ID: {material_id}")

        # Build and train the model
        model = build_and_train_model(X_train, y_train, model_type)

        # Forecast for the next time id (Time id 37)
        forecast_scaled = model.predict(X_train[-1:])  # Use the last time step of the training data for forecasting
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()[0]

        # Get actual Quantity value for Time id 37
        actual_quantity = test_data['Quantity'].values[0]

        # Evaluate the model for the current Material Id
        mape = evaluate_model(np.array([forecast]), np.array([actual_quantity]))
        print(f"MAPE: {mape}")

        # Append to forecast DataFrame
        forecast_df = forecast_df.append({
            'Material_Id': material_id,
            'Model': model_type,
            'MAPE': mape
        }, ignore_index=True)

# Display the forecast DataFrame with MAPE values
print(forecast_df)


Evaluating models for Material ID: 1311
No training or test data available for Material ID: 1311. Skipping evaluation.
Evaluating models for Material ID: 1421
Evaluating SimpleRNN model for Material ID: 1421


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
MAPE: 29.41736380259196
Evaluating GRU model for Material ID: 1421


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
MAPE: 20.944770177205406
Evaluating LSTM model for Material ID: 1421


...

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
MAPE: 7.442879676818848
Evaluating LSTM model for Material ID: 32814


C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({
C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
MAPE: 15.804100036621094
   Material_Id      Model        MAPE
0         1421  SimpleRNN   29.417364
1         1421        GRU   20.944770
2         1421       LSTM   31.219498
3         1610  SimpleRNN  166.767693
4         1610        GRU  184.420967
5         1610       LSTM  225.077987
6         6464  SimpleRNN   44.166856
7         6464        GRU   35.091771
8         6464       LSTM   32.167710
9         7263  SimpleRNN    9.045949
10        7263        GRU    9.225214
11        7263       LSTM    9.738889
12        8710  SimpleRNN   19.969635
13        8710        GRU   28.007360
14        8710       LSTM   27.507126
15       12481  SimpleRNN   79.360092
16       12481        GRU   71.992758
17       12481       LSTM   51.605442
18       25056  SimpleRNN   12.982633
19       25056        GRU    4.787560
20       25056       LSTM    3.973817
21       30765  SimpleRNN    3.150191
22       30765        GRU    3.577595
23       30765       LST

C:\Users\sohan\AppData\Local\Temp\ipykernel_11896\1359943296.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forecast_df = forecast_df.append({


In [11]:
forecast_df.to_csv('by4.csv', index=False)
